In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

2.2. CLIP-based Semantic Segmentation
Recently, the Contrastive Language-Image Pretraining
(CLIP) model [51] has been adopted in semantic segmen-
tation tasks thanks to the generalized knowledge learned
from a large corpus of image-text pairs. Given the gener-
alization capability, a number of zero-shot/open-vocabulary
approaches [17, 22, 34, 38, 47, 52, 67, 70, 71] exploit CLIP to
segment the classes which are unseen during training. How-
ever, these methods still require mask annotations during
training. To minimize the annotation effort, CLIP has also
been adopted to improve unsupervised methods [24, 58, 81].
Nevertheless, the segmentation performance is still unsat-
isfactory and is not desired for further applications. On
the other hand, CLIP has also been utilized to benefit
WSSS [42, 49, 65, 69, 72]. These works mainly focus on
designing text prompts or prompt learning techniques for
the text encoder. However, they either consider only the
foreground class prompts, or rely on general background
prompts defined by additional manual efforts and heuris-
tic human knowledge. Moreover, such manually-defined
prompts may not fully exploit the knowledge in the CLIP
latent space. In contrast, with no need for any manual
efforts, our proposed SemPLeS framework automatically
learns prompts embedded with class-associated semantic
knowledge discovered from the CLIP latent space.

In [26]:
# Define model parameters
batch_size = 2
channels = 3
height = 64
width = 64
prompt_dim = 512

# Create dummy inputs
X_f = torch.randn(batch_size, channels, height, width)  # Foreground image
X_b = torch.randn(batch_size, channels, height, width)  # Background image
t_k = torch.randn(batch_size, prompt_dim)  # Text prompt embeddings
p_k = torch.randn(batch_size, prompt_dim)  # Learnable background prompts

print(X_f.shape)
print(X_b.shape)
print(t_k.shape)
print(p_k.shape)

# Define ContrastivePromptLearning class
class ContrastivePromptLearning(nn.Module):
    def __init__(self, image_encoder, text_encoder, temperature=0.07):
        super().__init__()
        self.E_I = image_encoder  # Image encoder
        self.E_T = text_encoder  # Text encoder
        self.temperature = temperature

    def forward(self, X_f, X_b, t_k, p_k):
        """
        Contrastive Prompt Learning to learn background prompts
        Args:
            X_f: Foreground image (B, C, H, W)
            X_b: Background image (B, C, H, W) 
            t_k: Text prompt embeddings (B, prompt_dim)
            p_k: Learnable background prompts (B, prompt_dim)
        """
        # Flatten images before passing to linear encoder
        batch_size = X_f.shape[0]
        X_f_flat = X_f.view(batch_size, -1)  # Flatten to (B, C*H*W)
        X_b_flat = X_b.view(batch_size, -1)  # Flatten to (B, C*H*W)
        
        # Get image embeddings using CLIP image encoder
        v_f = self.E_I(X_f_flat)  # Foreground image embedding
        v_b = self.E_I(X_b_flat)  # Background image embedding
        
        # Get text embeddings using CLIP text encoder
        u_t = self.E_T(t_k)  # Text prompt embedding
        u_b = self.E_T(p_k)  # Background prompt embedding

        # Compute cosine similarities scaled by temperature
        sim_f_t = F.cosine_similarity(v_f, u_t, dim=-1) / self.temperature
        sim_b_p = F.cosine_similarity(v_b, u_b, dim=-1) / self.temperature
        sim_f_p = F.cosine_similarity(v_f, u_b, dim=-1) / self.temperature
        sim_b_t = F.cosine_similarity(v_b, u_t, dim=-1) / self.temperature

        # Compute contrastive loss
        L_contrast = -torch.log(torch.exp(sim_f_t) / (torch.exp(sim_f_t) + torch.exp(sim_f_p))) \
                    -torch.log(torch.exp(sim_b_p) / (torch.exp(sim_b_p) + torch.exp(sim_b_t)))
        
        return L_contrast.mean()


torch.Size([2, 3, 64, 64])
torch.Size([2, 3, 64, 64])
torch.Size([2, 512])
torch.Size([2, 512])


In [18]:


# Initialize model 
model = ContrastivePromptLearning(image_encoder, text_encoder)

# Forward pass
loss = model(X_f, X_b, t_k, p_k)

# Check output
assert isinstance(loss, torch.Tensor)
assert loss.dim() == 0  # scalar
assert not torch.isnan(loss)
assert not torch.isinf(loss)

print("ContrastivePromptLearning test passed!")

ContrastivePromptLearning test passed!


In [34]:
import PIL.Image
import requests
from torchvision import transforms
from pathlib import Path

# Create dummy encoders
image_encoder = nn.Linear(64 * 64 * 3, prompt_dim)  # Changed from 3 * 64 * 64 to match flattened input size
text_encoder = nn.Linear(prompt_dim, prompt_dim)

# Load and preprocess the image
image_path = Path("background.png")
image = PIL.Image.open(image_path)
print(image.shape)

# Define image transforms
preprocess = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Preprocess image
img_tensor = preprocess(image).unsqueeze(0)  # Add batch dimension

# Create dummy background image by slightly perturbing original
background = img_tensor + 0.1 * torch.randn_like(img_tensor)

# Text prompt
text_prompt = "Photo of a train"
text_embedding = torch.randn(1, prompt_dim)  # Simulate text embedding
background_prompt = torch.randn(1, prompt_dim)  # Learnable background prompt

# Forward pass with real data
loss = model(img_tensor, background, text_embedding, background_prompt)

print(f"Loss on real data: {loss.item():.4f}")

AttributeError: 'PngImageFile' object has no attribute 'shape'

In [6]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer

# Load CLIP text encoder and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# Text encoder using CLIP
class CLIPTextEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = text_model
        self.tokenizer = tokenizer
        
    def forward(self, text):
        if isinstance(text, str):
            # Tokenize and encode text
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
            outputs = self.model(**inputs)
            text_features = outputs.pooler_output
        else:
            # If input is already an embedding tensor, return as is
            text_features = text
            
        return text_features

# Initialize text encoder
text_encoder = CLIPTextEncoder()

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

C:\Users\craft\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Load CLIP image encoder
from transformers import CLIPVisionModel

image_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# Image encoder using CLIP
class CLIPImageEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = image_model
        
    def forward(self, image):
        if isinstance(image, torch.Tensor):
            # Process image tensor
            outputs = self.model(image)
            image_features = outputs.pooler_output
        else:
            # If input is already an embedding tensor, return as is
            image_features = image
            
        return image_features

# Initialize image encoder
image_encoder = CLIPImageEncoder()

In [12]:
# Implement Contrastive Prompt Learning loss
class ContrastivePromptLoss(nn.Module):
    def __init__(self, lambda_T=0.5):
        super().__init__()
        self.lambda_T = lambda_T
        
    def forward(self, u_bk, v_bk, u_fk):
        """
        Args:
            u_bk: Background prompt embedding from text encoder (B, prompt_dim)
            v_bk: Background image embedding from image encoder (B, prompt_dim) 
            u_fk: Text label embedding from text encoder (B, prompt_dim)
        """
        # Compute similarities using cosine similarity
        sim_bg = F.cosine_similarity(u_bk, v_bk, dim=-1)
        sim_text = F.cosine_similarity(u_bk, u_fk, dim=-1)
        
        # Background alignment loss (LIprompt)
        L_I_prompt = -torch.log(sim_bg)
        
        # Text dissimilarity loss (LTprompt) 
        L_T_prompt = -torch.log(1 - sim_text)
        
        # Total loss
        L_prompt = L_I_prompt + self.lambda_T * L_T_prompt
        
        return L_prompt.mean()

# Initialize loss
prompt_loss = ContrastivePromptLoss(lambda_T=0.5)

# Define background prompt and embeddings for training
batch_size = 2
prompt_dim = 512

# Initialize learnable background prompt
background_prompt = nn.Parameter(torch.randn(batch_size, prompt_dim))

# Create dummy background image tensor (B, C, H, W)
background = torch.randn(batch_size, 3, 64, 64)

# Create dummy text embedding tensor
text_embedding = torch.randn(batch_size, prompt_dim)

# Move models to CUDA
image_encoder = image_encoder.cuda()
text_encoder = text_encoder.cuda()
prompt_loss = prompt_loss.cuda()

# Move tensors to CUDA
background_prompt = background_prompt.cuda()
background = background.cuda()
text_embedding = text_embedding.cuda()

# Get embeddings from encoders
u_bk = text_encoder(background_prompt)  # Background prompt embedding
v_bk = image_encoder(background)  # Background image embedding 
u_fk = text_encoder(text_embedding)  # Text label embedding

# Compute prompt learning loss
prompt_learning_loss = prompt_loss(u_bk, v_bk, u_fk)

print(f"Prompt learning loss: {prompt_learning_loss.item():.4f}")

RuntimeError: The size of tensor a (5) must match the size of tensor b (50) at non-singleton dimension 1